In [7]:
#pip install pdf2image
#!pip install opencv-python

In [3]:
import pdf2image

In [8]:
from pdf2image import convert_from_path

pdfs = r"Invoice Copies/2.pdf"
pages = convert_from_path(pdfs, 350)

i = 1
for page in pages:
    image_name = "images/Page_" + str(i) + ".jpg"  
    page.save(image_name, "JPEG")
    i = i+1

FileNotFoundError: [Errno 2] No such file or directory: 'images/Page_1.jpg'